<h2>Virtual Imaging Techniques for 4D-STEM data</h2>
<strong>Jinseok Ryu, Ingyu Yoo<br></strong>
<strong>Electron Microscopy and Spectroscopy Lab.<br></strong>
<strong>Department of Materials Science and Engineering<br></strong>
<strong>Seoul National University<br><br></strong>

<strong>last update : 20221207</strong>

In [ ]:
from virtual_imaging import sstem_python
import tifffile
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import tkinter.filedialog as tkf
import time
plt.rcParams['font.family'] = 'Cambria'

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

datatype = "float32"
f_shape = [256, 256, 128, 128] # the shape of the 4D-STEM data [scanning_y, scanning_x, DP_y, DP_x]
o_shape = [f_shape[0], f_shape[1], f_shape[2]+2, f_shape[3]]

In [ ]:
if raw_adr[-3:] == "raw":
    f_stack = load_binary_4D_stack(raw_adr, datatype, o_shape, f_shape, log_scale=False)
    f_stack = np.flip(f_stack, axis=2)
    f_stack = np.nan_to_num(f_stack)
    
elif raw_adr[-3:] == "tif" or raw_adr[:-4] == "tiff":
    f_stack = tifffile.imread(raw_adr)
    
else:
    print("The format of the file is not supported here")
    
print(f_stack.shape)
print(f_stack.min(), f_stack.max())
print(f_stack.mean())

f_stack = f_stack.clip(min=0.0)

In [ ]:
scan_per_pixel = 0.11272 # Å
mrad_per_pixel = 0.40877 # mrad
test = sstem_python(f_stack.copy(), scan_per_pixel, mrad_per_pixel)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test.intensity_integration_map, cmap="inferno")
ax.axis("off")
plt.show()

In [ ]:
test.spike_remove(percent_thresh=99.9, mode="upper", apply_remove=True)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test.spike_replaced, cmap="inferno")
ax.axis("off")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test.original_pacbed, cmap="jet")
ax.axis("off")
plt.show()

In [ ]:
test.find_center()
print(test.ct)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test.original_pacbed, cmap="jet")
ax.axis("off")
ax.scatter(test.ct[1], test.ct[0], s=15, c="k")
fig.tight_layout()
plt.show()

In [ ]:
test.disk_extract(buffer_size=1)
print(test.c_stack.shape)
print(test.c_ct)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test.c_pacbed, cmap="jet")
ax.scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax.axis("off")
plt.show()
print(test.c_pacbed.shape)
print(test.least_R)

In [ ]:
semiangle = test.least_R*mrad_per_pixel
BF_det = np.array([0, semiangle])
ADF_det = np.array([semiangle, semiangle*2])

print(BF_det, ADF_det)

test.virtual_stem(BF_det, ADF_det)

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0][0].imshow(test.original_pacbed, cmap="jet")
ax[0][0].imshow(test.BF_detector, cmap="gray", alpha=0.5)
ax[0][0].scatter(test.ct[1], test.ct[0], s=15, c="k")
ax[0][0].set_title("BF detector")
ax[0][0].axis("off")
ax[0][1].imshow(test.BF_stem, cmap="inferno", origin="lower")
ax[0][1].set_title("BF-STEM image")
ax[0][1].axis("off")
ax[1][0].imshow(test.original_pacbed, cmap="jet")
ax[1][0].imshow(test.ADF_detector, cmap="gray", alpha=0.5)
ax[1][0].scatter(test.ct[1], test.ct[0], s=15, c="k")
ax[1][0].set_title("ADF detector")
ax[1][0].axis("off")
ax[1][1].imshow(test.ADF_stem, cmap="inferno", origin="lower")
ax[1][1].set_title("ADF-STEM image")
ax[1][1].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
test.DPC(correct_rotation=True, n_theta=100, hpass=0.05, lpass=0.05)

print("optimized angle =", test.c_theta*180/np.pi)
fig, ax = plt.subplots(1, 4, figsize=(28, 7))
ax[0].imshow(test.ADF_stem, cmap="inferno", origin="lower")
ax[0].axis("off")
ax[1].imshow(test.E_field_y, cmap="gray", origin="lower")
ax[1].axis("off")
ax[2].imshow(test.E_field_x, cmap="gray", origin="lower")
ax[2].axis("off")
ax[3].imshow(test.E_mag, cmap="inferno", origin="lower")
ax[3].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
RY, RX = np.indices(test.c_shape[:2])

fig, ax = plt.subplots(1, 3, figsize=(30, 10))
ax[0].imshow(test.ADF_stem, cmap="gray", origin="lower")
ax[0].quiver(RX.flatten(), RY.flatten(), test.E_field_x.flatten(), test.E_field_y.flatten(), color=cm.jet(mcolors.Normalize()(test.E_mag.flatten())))
ax[0].axis("off")
#ax[1].imshow(test.ADF_stem, cmap="gray")
ax[1].imshow(test.charge_density, cmap="RdBu_r", origin="lower")
ax[1].axis("off")
#ax[2].imshow(test.ADF_stem, cmap="gray")
ax[2].imshow(test.potential, cmap="inferno", origin="lower")
ax[2].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
tifffile.imwrite("vHAADF.tif", test.ADF_stem)
tifffile.imwrite("dDPC.tif", test.charge_density)
tifffile.imwrite("iDPC.tif", test.potential)

In [ ]:
alpha, beta = np.cos(test.c_theta*180/np.pi), np.sin(test.c_theta*180/np.pi)
M = np.array([[alpha, beta, (1-alpha)*test.c_ct[1]-beta*test.c_ct[0]], 
              [-beta, alpha, beta*test.c_ct[1]+(1-alpha)*test.c_ct[0]]])

rotated_pacbed = rotation(test.c_pacbed, M)
ri = radial_indices(test.c_pacbed.shape, [0, test.cropped_size], 1, center=test.c_ct)
mirror_plane = np.zeros(rotated_pacbed.shape)
mirror_plane[:, int(test.c_ct[1])] = 1
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.multiply(test.c_pacbed, ri), cmap="jet")
ax[0].imshow(mirror_plane, cmap="gray", alpha=0.2)
ax[0].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[0].axis("off")
ax[1].imshow(np.multiply(rotated_pacbed, ri), cmap="jet")
ax[1].imshow(mirror_plane, cmap="gray", alpha=0.2)
ax[1].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[1].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
start = time.process_time()
rot_angle = 3
rotated_stack, rot_sym_img, mir_sym_img = test.symmetry_evaluation(rot_angle, also_mirror=True)
print("%d seconds have passed"%(time.process_time()-start))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(rot_sym_img, cmap="inferno", origin="lower")
ax[0].set_title("Rotation angle: {}".format(rot_angle))
ax[0].axis("off")
ax[1].imshow(mir_sym_img, cmap="inferno", origin="lower")
ax[1].set_title("Mirror angle: {}".format(rot_angle))
ax[1].axis("off")
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0][0].imshow(test.c_stack[0, 0], cmap="jet")
ax[0][0].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[0][0].set_title("original CBED")
ax[0][0].axis("off")
ax[0][1].imshow(rotated_stack[0, 0], cmap="jet")
ax[0][1].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[0][1].set_title("rotated CBED (angle=%d)"%rot_angle)
ax[0][1].axis("off")
ax[1][0].imshow(test.c_stack[5, 5], cmap="jet")
ax[1][0].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[1][0].set_title("another original CBED")
ax[1][0].axis("off")
ax[1][1].imshow(rotated_stack[5, 5], cmap="jet")
ax[1][1].scatter(test.c_ct[1], test.c_ct[0], s=15, c="k")
ax[1][1].set_title("rotated CBED (angle=%d)"%rot_angle)
ax[1][1].axis("off")
fig.tight_layout()
plt.show()